In [1]:
# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('SparkCleaning').getOrCreate()

23/03/13 14:37:33 WARN Utils: Your hostname, MacBook-Air-di-Emanuele.local resolves to a loopback address: 127.0.0.1; using 10.0.0.197 instead (on interface en0)
23/03/13 14:37:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/13 14:37:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
df_spark = spark.read.csv("../datasets/imdb/validation_hidden.csv", inferSchema=True, header=True)
df_spark.repartition(4)

DataFrame[_c0: int, tconst: string, primaryTitle: string, originalTitle: string, startYear: string, endYear: string, runtimeMinutes: string, numVotes: double]

In [4]:
df_spark.show()

23/03/13 14:37:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes
 Schema: _c0, tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes
Expected: _c0 but found: 
CSV file: file:///Users/manu/PycharmProjects/Big-Data-Project/datasets/imdb/validation_hidden.csv
+---+---------+--------------------+-------------------+---------+-------+--------------+--------+
|_c0|   tconst|        primaryTitle|      originalTitle|startYear|endYear|runtimeMinutes|numVotes|
+---+---------+--------------------+-------------------+---------+-------+--------------+--------+
|  0|tt0003740|             Cabiria|               null|     1914|     \N|           148|  3452.0|
|  1|tt0008663|     A Man There Was|        Terje Vigen|     1917|     \N|            65|  1882.0|
|  3|tt0010307|           J'accuse!|               null|     1919|     \N|           166|  1692.0|
| 18|tt00

In [5]:
df_spark.count()

955

Starting to do stuff

In [6]:
import pandas as pd
import os

Count how many "train-i" files we have

In [7]:
dir_path = "../datasets/imdb/"

# Get a list of all the files in the directory
files = os.listdir(dir_path)

# Filter the list to include only files that start with "train-" and have a ".csv" extension
train_files = [f for f in files if f.startswith("train-") and f.endswith(".csv")]

# Count the number of files
num_train_files = len(train_files)

print("Number of train files: {}".format(num_train_files))

Number of train files: 8


print shape of training files, concatenate all the files, and print shape of big_file

In [8]:
# Create an empty list to store the DataFrames
dfs = []

# Loop through the file names and read each file into a DataFrame
for file in train_files:
    file_name = f"../datasets/imdb/{file}"
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    dfs.append(df)

# Loop through the DataFrames and print the shape of each one
for i, df in enumerate(dfs):
    num_rows = df.count()
    num_cols = len(df.columns)
    print("Shape of df_train_{}: {} rows, {} columns".format(i+1, num_rows, num_cols))

# Union all the DataFrames into one big DataFrame
big_df = dfs[0]
for df in dfs[1:]:
    big_df = big_df.union(df)

#Drop columns _c0
big_df = big_df.drop("_c0")

num_rows = big_df.count()
num_cols = len(big_df.columns)
print(f"\nShape of big_df: {num_rows} rows, {num_cols} columns")

Shape of df_train_1: 1001 rows, 9 columns
Shape of df_train_2: 1035 rows, 9 columns
Shape of df_train_3: 974 rows, 9 columns
Shape of df_train_4: 1049 rows, 9 columns
Shape of df_train_5: 963 rows, 9 columns
Shape of df_train_6: 951 rows, 9 columns
Shape of df_train_7: 993 rows, 9 columns
Shape of df_train_8: 993 rows, 9 columns

Shape of big_df: 7959 rows, 8 columns


In [9]:
big_df.show()

+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+
|   tconst|        primaryTitle|       originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|
+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+
|tt0013257|               Häxan|               Häxan|     1922|     \N|            91| 13679.0| true|
|tt0013556|          Robin Hood|                null|     1922|     \N|           143|  2178.0| true|
|tt0014341|     Our Hospitality|     Our Hospitality|     1923|     \N|            65| 10911.0| true|
|tt0014538|          Three Ages|                null|     1923|     \N|            63|  4312.0| true|
|tt0017925|         The General|                null|     1926|     \N|            67| 87784.0| true|
|tt0018192|            Napoleon|Napoléon vu par A...|     1927|     \N|           330|  7622.0| true|
|tt0018526|          Úndérwớrld|                null|     1927|     \N|           

In [10]:
#function to count number of distinct(unique) movies and total movies in dataset. To avoid duplicates number should be equal
def count_total_unique(df):
    unique_movies = df.select("tconst").distinct()
    total_movies = df.select("tconst")

    print("total movies:", total_movies.count())
    print("unique movies:", unique_movies.count())

count_total_unique(big_df)

total movies: 7959


unique movies: 7959


Merging with writing.json

In [11]:
import json
with open("../datasets/imdb/writing.json") as f:
    data = json.load(f)

#convert to pandas
writing_df = pd.DataFrame(data)

#taking care of multiple writers and adding them to a list
writing_df = writing_df.groupby('movie')['writer'].apply(list).reset_index()

#convert to spark
writing_df = spark.createDataFrame(writing_df)

# Rename the "movie" column in the JSON file to "tconst"
writing_df = writing_df.withColumnRenamed("movie", "tconst")

# Join the two DataFrames based on the "tconst"
merged_df = big_df.join(writing_df, on='tconst', how='left')

merged_df.show()

+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+--------------------+
|   tconst|        primaryTitle|       originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|              writer|
+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+--------------------+
|tt0018526|          Úndérwớrld|                null|     1927|     \N|            80|  2912.0| true|[nm0299153, nm000...|
|tt0031334|   The Four Feathers|                null|     1939|     \N|           129|  5917.0| true|[nm0556571, nm079...|
|tt0025410|  Little Miss Marker|  Little Miss Marker|     1934|     \N|            80|  1277.0| true|[nm0750357, nm051...|
|tt0013257|               Häxan|               Häxan|     1922|     \N|            91| 13679.0| true|         [nm0159725]|
|tt0027914|     Love on the Run|                null|     1936|     \N|            80|  1219.0|false|[nm0536941, nm078...|
|tt0029146|The L

In [12]:
count_total_unique(merged_df)

total movies: 7959
unique movies: 7959


Merging with directing.json

In [13]:
import json
with open("../datasets/imdb/directing.json") as f:
    data = json.load(f)

#convert to pandas
director_df = pd.DataFrame(data)

#taking care of multiple writers and adding them to a list
director_df = director_df.groupby('movie')['director'].apply(list).reset_index()

#convert to spark
director_df = spark.createDataFrame(director_df)

# Rename the "movie" column in the JSON file to "tconst"
director_df = director_df.withColumnRenamed("movie", "tconst")

# Join the two DataFrames based on the "tconst"
merged_df = merged_df.join(director_df, on='tconst', how='left')
merged_df.show()

+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+--------------------+--------------------+
|   tconst|        primaryTitle|       originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|              writer|            director|
+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+--------------------+--------------------+
|tt0018526|          Úndérwớrld|                null|     1927|     \N|            80|  2912.0| true|[nm0299153, nm000...|[nm0903049, nm074...|
|tt0031334|   The Four Feathers|                null|     1939|     \N|           129|  5917.0| true|[nm0556571, nm079...|         [nm0466113]|
|tt0025410|  Little Miss Marker|  Little Miss Marker|     1934|     \N|            80|  1277.0| true|[nm0750357, nm051...|         [nm0355284]|
|tt0013257|               Häxan|               Häxan|     1922|     \N|            91| 13679.0| true|         [nm0159725]|         [nm01

In [14]:
count_total_unique(merged_df)

total movies: 7959
unique movies: 7959


In [15]:
big_df.show()

+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+
|   tconst|        primaryTitle|       originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|
+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+
|tt0013257|               Häxan|               Häxan|     1922|     \N|            91| 13679.0| true|
|tt0013556|          Robin Hood|                null|     1922|     \N|           143|  2178.0| true|
|tt0014341|     Our Hospitality|     Our Hospitality|     1923|     \N|            65| 10911.0| true|
|tt0014538|          Three Ages|                null|     1923|     \N|            63|  4312.0| true|
|tt0017925|         The General|                null|     1926|     \N|            67| 87784.0| true|
|tt0018192|            Napoleon|Napoléon vu par A...|     1927|     \N|           330|  7622.0| true|
|tt0018526|          Úndérwớrld|                null|     1927|     \N|           

In [16]:
#function to count number of distinct(unique) movies and total movies in dataset. To avoid duplicates number should be equal
def count_total_unique(df):
    unique_movies = df.select("tconst").distinct()
    total_movies = df.select("tconst")

    print("total movies:", total_movies.count())
    print("unique movies:", unique_movies.count())

count_total_unique(big_df)

total movies: 7959
unique movies: 7959


Merging with writing.json

In [48]:
import json
with open("../datasets/imdb/writing.json") as f:
    data = json.load(f)

#convert to pandas
writing_df = pd.DataFrame(data)

#taking care of multiple writers and adding them to a list
writing_df = writing_df.groupby('movie')['writer'].apply(list).reset_index()

#convert to spark
writing_df = spark.createDataFrame(writing_df)

# Rename the "movie" column in the JSON file to "tconst"
writing_df = writing_df.withColumnRenamed("movie", "tconst")

# Join the two DataFrames based on the "tconst"
merged_df = big_df.join(writing_df, on='tconst', how='left')

merged_df.show()

+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+--------------------+
|   tconst|        primaryTitle|       originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|              writer|
+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+--------------------+
|tt0018526|          Úndérwớrld|                null|     1927|     \N|            80|  2912.0| true|[nm0299153, nm000...|
|tt0031334|   The Four Feathers|                null|     1939|     \N|           129|  5917.0| true|[nm0556571, nm079...|
|tt0025410|  Little Miss Marker|  Little Miss Marker|     1934|     \N|            80|  1277.0| true|[nm0750357, nm051...|
|tt0013257|               Häxan|               Häxan|     1922|     \N|            91| 13679.0| true|         [nm0159725]|
|tt0027914|     Love on the Run|                null|     1936|     \N|            80|  1219.0|false|[nm0536941, nm078...|
|tt0029146|The L

In [49]:
count_total_unique(merged_df)

total movies: 7959
unique movies: 7959


Merging with directing.json

In [50]:
import json
with open("../datasets/imdb/directing.json") as f:
    data = json.load(f)

#convert to pandas
director_df = pd.DataFrame(data)

#taking care of multiple writers and adding them to a list
director_df = director_df.groupby('movie')['director'].apply(list).reset_index()

#convert to spark
director_df = spark.createDataFrame(director_df)

# Rename the "movie" column in the JSON file to "tconst"
director_df = director_df.withColumnRenamed("movie", "tconst")

# Join the two DataFrames based on the "tconst"
merged_df = merged_df.join(director_df, on='tconst', how='left')
merged_df.show()

+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+--------------------+--------------------+
|   tconst|        primaryTitle|       originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|              writer|            director|
+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+--------------------+--------------------+
|tt0018526|          Úndérwớrld|                null|     1927|     \N|            80|  2912.0| true|[nm0299153, nm000...|[nm0903049, nm074...|
|tt0031334|   The Four Feathers|                null|     1939|     \N|           129|  5917.0| true|[nm0556571, nm079...|         [nm0466113]|
|tt0025410|  Little Miss Marker|  Little Miss Marker|     1934|     \N|            80|  1277.0| true|[nm0750357, nm051...|         [nm0355284]|
|tt0013257|               Häxan|               Häxan|     1922|     \N|            91| 13679.0| true|         [nm0159725]|         [nm01

In [51]:
count_total_unique(merged_df)

total movies: 7959


unique movies: 7959


In [52]:
from pyspark.sql.functions import regexp_replace, col, lower

In [64]:
def pre_processing(df):
    """
    This method takes in input a dataframe and preprocess the columns
    :return:
    """

    df = df.withColumn('primaryTitle', lower(col('primaryTitle')))
    df = df.withColumn('originalTitle', lower(col('originalTitle')))

    characters_to_substitute = [('á', 'a'), ('é', 'e'), ('ớ', 'o'), ('ú', 'u')]

    for character in characters_to_substitute:
        df = df.withColumn('primaryTitle', regexp_replace(col('primaryTitle'), character[0], character[1]))
        df = df.withColumn('originalTitle', regexp_replace(col('originalTitle'), character[0], character[1]))

    return df

In [79]:
"""
Load the validation dataset
"""

validation = spark.read.csv('../datasets/imdb/validation_hidden.csv', inferSchema=True, header=True)

In [85]:
"""
Loading the test dataset
"""
test = spark.read.csv('../datasets/imdb/test_hidden.csv', inferSchema=True, header=True)

In [86]:
merged_df = pre_processing(merged_df)
validation = pre_processing(validation)
test = pre_processing(test)

In [87]:
test.show()

23/03/13 15:37:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes
 Schema: _c0, tconst, primaryTitle, originalTitle, startYear, endYear, runtimeMinutes, numVotes
Expected: _c0 but found: 
CSV file: file:///Users/manu/PycharmProjects/Big-Data-Project/datasets/imdb/test_hidden.csv
+---+---------+--------------------+-------------------+---------+-------+--------------+--------+
|_c0|   tconst|        primaryTitle|      originalTitle|startYear|endYear|runtimeMinutes|numVotes|
+---+---------+--------------------+-------------------+---------+-------+--------------+--------+
| 22|tt0014972| he who gets slapped|he who gets slapped|     1924|     \N|            95|  3654.0|
| 23|tt0015016|      the iron horse|               null|     1924|     \N|           150|  2136.0|
| 26|tt0015174|die nibelungen: k...|               null|     1924|     \N|           129|  4341.0|
| 28|tt0015214|